[View in Colaboratory](https://colab.research.google.com/github/nikhilbhatewara/TeradataChallenge2018/blob/master/EDA.ipynb)

#Setup Commands

In [0]:
#!kill -9 -1

In [1]:
%%time
import pandas as pd
import numpy as np
import os
import scipy.stats as sts
!pip install xlrd
import io
!pip install mlxtend
# for Box-Cox Transformation
# for min_max scaling
from mlxtend.preprocessing import minmax_scaling
# plotting modules
import seaborn as sns
import matplotlib.pyplot as plt

Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 112kB 2.7MB/s 
Looking in indexes: https://pypi.org/simple, https://legacy.pypi.org/simple
    100% |████████████████████████████████| 1.3MB 6.4MB/s 
CPU times: user 103 ms, sys: 57 ms, total: 160 ms
Wall time: 5.47 s


In [0]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from xgboost import XGBRegressor
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.linear_model import LassoLarsCV

In [2]:
# Install a Drive FUSE wrapper.
# https://github.com/astrada/google-drive-ocamlfuse
!apt-get install -y -qq software-properties-common python-software-properties module-init-tools
!add-apt-repository -y ppa:alessandro-strada/ppa 2>&1 > /dev/null
!apt-get update -qq 2>&1 > /dev/null
!apt-get -y install -qq google-drive-ocamlfuse fuse

Preconfiguring packages ...
Selecting previously unselected package cron.
(Reading database ... 18298 files and directories currently installed.)
Preparing to unpack .../00-cron_3.0pl1-128ubuntu5_amd64.deb ...
Unpacking cron (3.0pl1-128ubuntu5) ...
Selecting previously unselected package libapparmor1:amd64.
Preparing to unpack .../01-libapparmor1_2.11.0-2ubuntu17.1_amd64.deb ...
Unpacking libapparmor1:amd64 (2.11.0-2ubuntu17.1) ...
Selecting previously unselected package libdbus-1-3:amd64.
Preparing to unpack .../02-libdbus-1-3_1.10.22-1ubuntu1_amd64.deb ...
Unpacking libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Selecting previously unselected package dbus.
Preparing to unpack .../03-dbus_1.10.22-1ubuntu1_amd64.deb ...
Unpacking dbus (1.10.22-1ubuntu1) ...
Selecting previously unselected package dirmngr.
Preparing to unpack .../04-dirmngr_2.1.15-1ubuntu8_amd64.deb ...
Unpacking dirmngr (2.1.15-1ubuntu8) ...
Selecting previously unselected package distro-info-data.
Preparing to unpack .../0


Creating config file /etc/apt/apt.conf.d/20auto-upgrades with new version

Creating config file /etc/apt/apt.conf.d/50unattended-upgrades with new version
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up dirmngr (2.1.15-1ubuntu8) ...
Setting up cron (3.0pl1-128ubuntu5) ...
Adding group `crontab' (GID 102) ...
Done.
update-rc.d: warning: start and stop actions are no longer supported; falling back to defaults
update-rc.d: warning: stop runlevel arguments (1) do not match cron Default-Stop values (none)
invoke-rc.d: could not determine current runlevel
invoke-rc.d: policy-rc.d denied execution of start.
Setting up libdbus-1-3:amd64 (1.10.22-1ubuntu1) ...
Setting up kmod (24-1ubuntu2) ...
Setting up libdbus-glib-1-2:amd64 (0.108-2) ...
Setting up python3-gi (3.24.1-2build1) ...
Setting up module-init-tools (24-1ubuntu2) ...
Setting up python3-software-properties (0.96.24.17) ...
Setting up dbus (1.10.22-1ubuntu1) ...
Setting

In [0]:
# Generate auth tokens for Colab
from google.colab import auth
auth.authenticate_user()

In [4]:
from oauth2client.client import GoogleCredentials
creds = GoogleCredentials.get_application_default()
import getpass
!google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret} < /dev/null 2>&1 | grep URL
vcode = getpass.getpass()
!echo {vcode} | google-drive-ocamlfuse -headless -id={creds.client_id} -secret={creds.client_secret}

Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
··········
Please, open the following URL in a web browser: https://accounts.google.com/o/oauth2/auth?client_id=32555940559.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive&response_type=code&access_type=offline&approval_prompt=force
Please enter the verification code: Access token retrieved correctly.


In [5]:
# Create a directory and mount Google Drive using that directory.
!mkdir -p drive
!google-drive-ocamlfuse drive

print ('Files in my project folder:')
!ls drive/OSURPYTHON/teradata

# Create a file in Drive.
!echo "This newly created file will appear in your Drive file list." > drive/OSURPYTHON/teradata/created.txt

Files in my project folder:
2013-2017 Bike Events.csv	       COPY OF NTA.xlsx
2013-2017 Bike MS Participants.csv     created.txt
2013-2017 National Team Activity.xlsx  donationsbike2013_clean.csv
2013 Bike Donations.csv		       donationsbike2014_clean.csv
2014 Bike Donations.csv		       donationsbike2015_clean.csv
2015 Bike Donations.csv		       donationsbike2016_clean.csv
2016 Bike Donations.csv		       donationsbike2017_clean.csv
2017 Bike Donations.csv		       eventsbike_clean.csv
affiliatecodes_clean.csv	       ntactivity_clean.csv
Affiliate_Codes.xlsx		       participantbike_clean.csv
BikeTeams.csv			       Working book.ipynb


Using pydrive to upload data in gdrive

In [0]:
# Install the PyDrive wrapper & import libraries.
# This only needs to be done once in a notebook.
!pip install -U -q PyDrive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# Authenticate and create the PyDrive client.
# This only needs to be done once in a notebook.
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)



#Importing Files

In [8]:
%%time
eventsbike = pd.read_csv("drive/OSURPYTHON/teradata/2013-2017 Bike Events.csv",engine="python")
teamsbike = pd.read_csv("drive/OSURPYTHON/teradata/BikeTeams.csv",engine="python")
participantbike = pd.read_csv("drive/OSURPYTHON/teradata/2013-2017 Bike MS Participants.csv",engine="python")
donationsbike2017 = pd.read_csv("drive/OSURPYTHON/teradata/2017 Bike Donations.csv",engine="python")
donationsbike2016 = pd.read_csv("drive/OSURPYTHON/teradata/2016 Bike Donations.csv",engine="python")
donationsbike2015 = pd.read_csv("drive/OSURPYTHON/teradata/2015 Bike Donations.csv",engine="python")
donationsbike2014 = pd.read_csv("drive/OSURPYTHON/teradata/2014 Bike Donations.csv",engine="python")
donationsbike2013 = pd.read_csv("drive/OSURPYTHON/teradata/2013 Bike Donations.csv",engine="python")
affiliatecodes= pd.read_excel("drive/OSURPYTHON/teradata/Affiliate_Codes.xlsx")
ntactivity =  pd.read_excel("drive/OSURPYTHON/teradata/COPY OF NTA.xlsx")


CPU times: user 618 ms, sys: 41 ms, total: 659 ms
Wall time: 1.85 s


Replace space with underscore to clean the column names

In [9]:
%%time
eventsbike.columns = eventsbike.columns.str.replace('\s+', '_') 
teamsbike.columns = teamsbike.columns.str.replace('\s+', '_') 
participantbike.columns = participantbike.columns.str.replace('\s+', '_')  # in case there are multiple white spaces
donationsbike2017.columns = donationsbike2017.columns.str.replace('\s+', '_')  # in case there are multiple white spaces
donationsbike2016.columns = donationsbike2016.columns.str.replace('\s+', '_')  # in case there are multiple white spaces
donationsbike2015.columns = donationsbike2015.columns.str.replace('\s+', '_')  
donationsbike2014.columns = donationsbike2014.columns.str.replace('\s+', '_')  
donationsbike2013.columns = donationsbike2013.columns.str.replace('\s+', '_')
affiliatecodes.columns = affiliatecodes.columns.str.replace('\s+', '_')
ntactivity.columns = ntactivity.columns.str.replace('\s+', '_')
 

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 648 µs


Dummy Date : '01/01/2000 00:00'
Dummy Zip: 999999

#Data Cleaning for Eventsbike

In [0]:
eventsbike.isnull().sum().plot(kind='bar')

In [0]:
eventsbike=eventsbike.dropna(subset = ['Event_Date'])

In [0]:

eventsbike.loc[116,'Street_Address'] = '100 CAMPUS DR'
eventsbike.loc[526,'Street_Address'] = '100 CAMPUS DR'
eventsbike.loc[35,'Zip_Code']= 93001.0
eventsbike.loc[53,'Zip_Code']= 94158.0
eventsbike.loc[51,'Zip_Code']= 94158.0
eventsbike.loc[38,'Zip_Code']= 89106.0
eventsbike.loc[216:217,'Zip_Code']= 70401.0
eventsbike.loc[221,'Zip_Code']= 70401.0
eventsbike.loc[305,'Zip_Code']= 48451.0
eventsbike.loc[297,'Zip_Code']= 49423.0
eventsbike.loc[296,'Zip_Code']= 48116.0
eventsbike.loc[294,'Zip_Code']= 48734.0
eventsbike.loc[380,'Zip_Code']= 28560.0
eventsbike.loc[386,'Zip_Code']= 8551.0
eventsbike.loc[619,'Zip_Code']= 999999.0
eventsbike.loc[542,'Zip_Code']= 16648.0
eventsbike.loc[544,'Zip_Code']= 16648.0
eventsbike.loc[584,'Zip_Code']= 38671.0
eventsbike.loc[567,'Zip_Code']= 37064.0
eventsbike.loc[631,'Zip_Code']= 84321.0

In [0]:
eventsbike['Zip_Code']=eventsbike['Zip_Code'].fillna(999999)

In [0]:
eventsbike['Event_Created_Date']=eventsbike['Event_Created_Date'].fillna( '01/01/2000 00:00' )

In [0]:
eventsbike['Security_Category_Name']=eventsbike['Security_Category_Name'].fillna('Unknown')
eventsbike['State']=eventsbike['State'].fillna('Unknown')
eventsbike['City']=eventsbike['City'].fillna('Unknown')
eventsbike['Street_Address']=eventsbike['Street_Address'].fillna('Unknown')

In [0]:

eventsbike['Security_Category_Name']=eventsbike['Security_Category_Name'].astype('category')
eventsbike['Event_Created_Date']=pd.to_datetime(eventsbike['Event_Created_Date'])
eventsbike['State']=eventsbike['State'].astype('category')
eventsbike['Zip_Code']=eventsbike['Zip_Code'].astype('int')
eventsbike['City']=eventsbike['City'].astype('category')
eventsbike['Street_Address']=eventsbike['Street_Address'].astype('category')



In [112]:
eventsbike.dtypes

Fiscal_Year                                    int64
Event_ID                                       int64
Security_Category_Name                      category
Public_Event_Name                             object
Event_Created_Date                    datetime64[ns]
Event_Date                                    object
Event_Goal($)                                  int64
Active_Registrations                           int64
Inactive_Registrations                         int64
Total_Fees_Paid                              float64
Total_of_All_Confirmed_Gifts($)              float64
Total_Online_Gifts($)                        float64
Teams                                          int64
Captains                                       int64
Average_Team_Size                              int64
Team_pages_with_custom_URL                     int64
Emails_Sent                                    int64
Personal_pages_with_custom_URL                 int64
Total_Offline_Confirmed_Gifts($)             f

**From FAQs**
Question 9a: We found that some of the events do not have event dates. What does this imply? Or just error in recording the data?

You would only find missing event dates in test events. Test events were created to test various functions and updates. Unfortunately, they are left in the system after being created


In [0]:
eventsbike['Public_Event_Name']=eventsbike['Public_Event_Name'].astype('category')


In [0]:
eventsbike['Internal_Event_Name']=eventsbike['Internal_Event_Name'].astype('category')

In [0]:
from google.colab import files

eventsbike.to_csv('eventsbike_clean.csv')

In [126]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'eventsbike_clean.csv'})
uploaded.SetContentFile('eventsbike_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1-BoY_cqSE3hhuxftgHwRG9nkxFNW607L


#Data Cleaning for teamsbike

In [0]:
teamsbike.isnull().sum().plot(kind='bar')


In [0]:
teamsbike=teamsbike.dropna(subset = ['Event_Date'])

In [0]:
teamsbike.isnull().sum().plot(kind='bar')

unknown years have been replaced as 0000

These 2 events do not have fiscal year but it can be imputed from the event date

In [0]:
teamsbike.loc[22706,['Fiscal_Year']]=2016
teamsbike.loc[29752,['Fiscal_Year']]=2017

In [0]:
teamsbike=teamsbike.fillna({'Team_Captain_Contact_ID':0,'Previous_Event_Fiscal_Year':0000,'Company':'Some LLP','Total_Confirmed_Gifts_in_Team_History($)':0})

Wrong data, cannot change it so have to delete it

In [0]:

teamsbike=teamsbike.drop(teamsbike.index[29752])

Contact ID has wrong value

In [0]:
teamsbike.loc[22706,['Team_Captain_Contact_ID']]=13263460

In [132]:
teamsbike.dtypes

Event_Type                                   object
Internal_Event_Name                          object
Event_ID                                      int64
Team_ID                                       int64
Team_Name                                    object
Team_Creation_Date                           object
Team_Captain_Contact_ID                      object
Captain_Email_Domain                         object
Team_Captain_Accept_Email                    object
Team_Division                                object
Company                                      object
Number_of_Participants                        int64
Total_Fees_Paid                             float64
Team_Total_Confirmed_($)                     object
Total_Online_Gifts($)                       float64
Total_Offline_Confirmed_Gifts($)            float64
Total_Offline_Unconfirmed_Gifts($)          float64
Team_Goal($)                                float64
Total_Confirmed_Gifts_in_Team_History($)    float64
Previous_Eve

In [0]:
teamsbike=teamsbike.fillna('Unknown')

In [0]:
teamsbike['Event_Type']=teamsbike['Event_Type'].astype('category')
teamsbike['Internal_Event_Name']=teamsbike['Internal_Event_Name'].astype('category')
teamsbike['Team_Name']=teamsbike['Team_Name'].astype('category')
teamsbike['Captain_Email_Domain']=teamsbike['Captain_Email_Domain'].astype('category')
teamsbike['Team_Division']=teamsbike['Team_Division'].astype('category')
teamsbike['Company']=teamsbike['Company'].astype('category')
teamsbike['Previous_Event_Internal_Name']=teamsbike['Previous_Event_Internal_Name'].astype('category')
teamsbike['Previous_Event_Team_Name']=teamsbike['Previous_Event_Team_Name'].astype('category')
teamsbike['Previous_Event_Team_Members']=teamsbike['Previous_Event_Team_Members'].astype('category')
teamsbike['Event_Date']=pd.to_datetime(teamsbike['Event_Date'],format="%m/%d/%Y %H:%M")
teamsbike['Team_Creation_Date']=pd.to_datetime(teamsbike['Team_Creation_Date'],format="%m/%d/%Y %H:%M")
teamsbike['Fiscal_Year']=teamsbike['Fiscal_Year'].astype('int')
teamsbike['Previous_Event_Fiscal_Year']=teamsbike['Previous_Event_Fiscal_Year'].astype('int')
teamsbike['Total_Confirmed_Gifts_in_Team_History($)']=teamsbike['Total_Confirmed_Gifts_in_Team_History($)'].astype('float')
teamsbike['Team_Total_Confirmed_($)']=teamsbike['Team_Total_Confirmed_($)'].astype('float')
teamsbike['Team_Captain_Contact_ID']=teamsbike['Team_Captain_Contact_ID'].astype('int')
teamsbike['Team_Captain_Accept_Email']=teamsbike['Team_Captain_Accept_Email'].astype('category')


In [0]:
from google.colab import files

teamsbike.to_csv('teamsbike_clean.csv')

In [18]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'teamsbike_clean.csv'})
uploaded.SetContentFile('teamsbike_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 15ZbuyaoIG7YRggsccjp2bybC7Z2BvkHF


#Data Cleaning for participantbike

In [0]:
participantbike.isnull().sum().plot(kind='bar')

In [0]:
participantbike.columns

In [0]:
participantbike=participantbike.dropna(subset = ['Event_Date'])

In [0]:
participantbike=participantbike.fillna({'Participant_Connection_to_MS':'Other','Participant_Employer':'Some_LLP','Team_ID':'0','Member_ID':'0','Team_Creation_Date':'01/01/2000 00:00'})

In [0]:
participantbike=participantbike.fillna('Unknown')

Since some member_ID contains "#_removed" and it is not known what _removed means, we are dropping this column

In [0]:
participantbike=participantbike.drop('Member_ID',axis=1)

In [0]:
#89871577_removed
#participantbike.loc[participantbike.Member_ID=="85130489_removed"]
#participantbike.loc[65459,['Member_ID']]=89871577
#participantbike.loc[65391,['Member_ID']]=85130489

In [0]:
participantbike[	'Security_Category_Name'	]=	participantbike[	'Security_Category_Name'	].astype('category')
participantbike[	 'Internal_Event_Name'	]=	participantbike[	 'Internal_Event_Name'	].astype('category')
participantbike[	 'Participation_Type_Name'	]=	participantbike[	 'Participation_Type_Name'	].astype('category')
participantbike[	 'Team_Name'	]=	participantbike[	 'Team_Name'	].astype('category')
participantbike[	 'Team_Division'	]=	participantbike[	 'Team_Division'	].astype('category')
participantbike[	 'Participant_Accept_Email'	]=	participantbike[	 'Participant_Accept_Email'	].astype('category')
participantbike[	 'Registration_Active_Status'	]=	participantbike[	 'Registration_Active_Status'	].astype('category')
participantbike[	 'Is_Team_Captain'	]=	participantbike[	 'Is_Team_Captain'	].astype('category')
participantbike[	 'Is_Secondary_Registration'	]=	participantbike[	 'Is_Secondary_Registration'	].astype('category')
participantbike[	 'Is_Prior_Participant'	]=	participantbike[	 'Is_Prior_Participant'	].astype('category')
participantbike[	 'Emails_Sent'	]=	participantbike[	 'Emails_Sent'	].astype('category')
participantbike[	 'Participant_Email_Status'	]=	participantbike[	 'Participant_Email_Status'	].astype('category')
participantbike[	 'Participant_Employer'	]=	participantbike[	 'Participant_Employer'	].astype('category')
participantbike[	 'Participant_Occupation'	]=	participantbike[	 'Participant_Occupation'	].astype('category')
participantbike[	 'Participant_Connection_to_MS'	]=	participantbike[	 'Participant_Connection_to_MS'	].astype('category')
participantbike[	 'Address_-_Participant_State/Province'	]=	participantbike[	 'Address_-_Participant_State/Province'	].astype('category')
participantbike[	 'Address_-_Participant_County'	]=	participantbike[	 'Address_-_Participant_County'	].astype('category')
participantbike[	 'Address_-_Participant_City'	]=	participantbike[	 'Address_-_Participant_City'	].astype('category')
participantbike[	 'Registration_Type'	]=	participantbike[	 'Registration_Type'	].astype('category')
participantbike[	 'Participant_Gender'	]=	participantbike[	 'Participant_Gender'	].astype('category')
participantbike[	 'Source_Code_Type'	]=	participantbike[	 'Source_Code_Type'	].astype('category')
participantbike[	 'Source_Code_Text'	]=	participantbike[	 'Source_Code_Text'	].astype('category')
participantbike[	 'Sub_Source_Code_Text']	=	participantbike[	 'Sub_Source_Code_Text']	.astype('category')

In [0]:
participantbike[	 'Team_ID'	]=	participantbike[	 'Team_ID'	].astype('float')
participantbike[	 'Contact_ID'	]=	participantbike[	 'Contact_ID'	].astype('float')
#participantbike[	 'Member_ID'	]=	participantbike[	 'Member_ID'	].astype('float')
participantbike[	 'Total_of_All_Confirmed_Gifts($)'	]=	participantbike[	 'Total_of_All_Confirmed_Gifts($)'	].astype('float')
participantbike[	 'Total_From_Participant($)'	]=	participantbike[	 'Total_From_Participant($)'	].astype('float')
participantbike[	 'Total_Not_From_Participant($)'	]=	participantbike[	 'Total_Not_From_Participant($)'	].astype('float')
participantbike[	 'Number_From_Participant'	]=	participantbike[	 'Number_From_Participant'	].astype('float')
participantbike[	 'Number_Not_From_Participant'	]=	participantbike[	 'Number_Not_From_Participant'	].astype('float')
participantbike[	 'Event_ID'	]=	participantbike[	 'Event_ID'	].astype('float')
participantbike[	 'Participant_Goal($)'	]=	participantbike[	 'Participant_Goal($)'	].astype('float')
participantbike[	 'Fiscal_Year'	]=	participantbike[	 'Fiscal_Year'	].astype('float')



In [0]:
participantbike[	 'Event_Date'	]=	pd.to_datetime(participantbike[	 'Event_Date'	],format="%m/%d/%Y %H:%M")
participantbike[	 'Team_Creation_Date'	]=	pd.to_datetime(participantbike[	 'Team_Creation_Date'	],format="%m/%d/%Y %H:%M")
participantbike[	 'Registration_Date'	]=	pd.to_datetime(participantbike[	 'Registration_Date'	],format="%m/%d/%Y %H:%M")


In [0]:
participantbike.isnull().sum().plot(kind='bar')

In [0]:
from google.colab import files

participantbike.to_csv('participantbike_clean.csv')

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'participantbike_clean.csv'})
uploaded.SetContentFile('participantbike_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

#Data Cleaning for affiliatecodes

In [0]:
affiliatecodes.isnull().sum().plot(kind='bar')

In [0]:

for col in affiliatecodes.columns:
  affiliatecodes[col] = affiliatecodes[col].astype('category')

In [0]:
from google.colab import files

affiliatecodes.to_csv('affiliatecodes_clean.csv')

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'affiliatecodes_clean.csv'})
uploaded.SetContentFile('affiliatecodes_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1w79M432WEVlGHI3z4oWlGeAf9U-3_t7n


#Data Cleaning for ntactivity

Since there are team names which are merged, we have to unmerged them and create a new file

Here the First Line represents consolidated revenue and no of participants for each time. this is not required in our data frame hence we will drop that rows

In [0]:
ntactivity = ntactivity.dropna(subset=['Event_Chapter'])


From the shape function we can observe that 107 rows have been dropped.

Here the assumption is the atleast 1 participant must be present in the team, hence we are replacing the  NAN value of 'number of team members' = 1

Also, contact id and mermber id have been replaced by 0.Zip code by 9999999 so as to keep the data type of columns consistent

Participant Connection of MS have been substituted as OTHER since we do not know if they have any connection or not and substituing "NONE" will not be appropriate

In [0]:
ntactivity=ntactivity.fillna({'Participant_Connection_to_MS':'Other','Zip_Code':'999999','Contact_ID':0,'Member_ID':0,'Contact_By_Email_OK?_(Y/N)':'N','Contact_By_Phone_OK?_(Y/N)':'N','Contact_By_Mail_OK?_(Y/N)':'N'})

In [0]:
teamcount=ntactivity.loc[:,['Number_of_Team_Members']].copy()
ntactivity=ntactivity.drop('Number_of_Team_Members',axis=1)
ntactivity=ntactivity.fillna('Unknown')


In [0]:
ntactivity.isnull().values.any()

False

In [0]:
ntactivity[	'National_Team_Name'	]=ntactivity[	'National_Team_Name'	].astype('category')
ntactivity[	'Event_Chapter'	]=ntactivity[	'Event_Chapter'	].astype('category')
ntactivity[	'Event_Category'	]=ntactivity[	'Event_Category'	].astype('category')
ntactivity[	'Event_Type'	]=ntactivity[	'Event_Type'	].astype('category')
ntactivity[	'Local_Team_Name'	]=ntactivity[	'Local_Team_Name'	].astype('category')
ntactivity[	'Event_Name'	]=ntactivity[	'Event_Name'	].astype('category')
ntactivity[	'Location'	]=ntactivity[	'Location'	].astype('category')
ntactivity[	'Corporate_Name'	]=ntactivity[	'Corporate_Name'	].astype('category')
ntactivity[	'Email_Domain'	]=ntactivity[	'Email_Domain'	].astype('category')
ntactivity[	'Bad_Email_Flag'	]=ntactivity[	'Bad_Email_Flag'	].astype('category')
ntactivity[	'Contact_By_Email_OK?_(Y/N)'	]=ntactivity[	'Contact_By_Email_OK?_(Y/N)'	].astype('category')
ntactivity[	'Phone_Type'	]=ntactivity[	'Phone_Type'	].astype('category')
ntactivity[	'Call_Status'	]=ntactivity[	'Call_Status'	].astype('category')
ntactivity[	'Contact_By_Phone_OK?_(Y/N)'	]=ntactivity[	'Contact_By_Phone_OK?_(Y/N)'	].astype('category')
ntactivity[	'City'	]=ntactivity[	'City'	].astype('category')
ntactivity[	'State/Province'	]=ntactivity[	'State/Province'	].astype('category')
ntactivity[	'Bad_Address_Flag'	]=ntactivity[	'Bad_Address_Flag'	].astype('category')
ntactivity[	'Contact_By_Mail_OK?_(Y/N)'	]=ntactivity[	'Contact_By_Mail_OK?_(Y/N)'	].astype('category')
ntactivity[	'Active_Flag'	]=ntactivity[	'Active_Flag'	].astype('category')
ntactivity[	'Deceased_Flag'	]=ntactivity[	'Deceased_Flag'	].astype('category')
ntactivity[	'Primary_Connection_To_MS'	]=ntactivity[	'Primary_Connection_To_MS'	].astype('category')


In [0]:
ntactivity[	'Revenue_Raised'	]=ntactivity[	'Revenue_Raised'	].astype('float')
ntactivity[	'Member_ID'	]=ntactivity[	'Member_ID'	].astype('int')
ntactivity[	'Contact_ID'	]=ntactivity[	'Contact_ID'	].astype('int')


In [0]:
ntactivity[	'Event_Start_Date'	]=	pd.to_datetime(ntactivity[	'Event_Start_Date'	],format="%m/%d/%Y %H:%M")


In [0]:
ntactivity = pd.concat([ntactivity, teamcount], axis=1)

In [0]:
ntactivity.columns

Index(['National_Team_Name', 'Event_Chapter', 'Event_Category', 'Event_Type',
       'Local_Team_Name', 'Event_Name', 'Event_Start_Date', 'Location',
       'Corporate_Name', 'Revenue_Raised', 'Member_ID', 'Contact_ID',
       'Email_Domain', 'Bad_Email_Flag', 'Contact_By_Email_OK?_(Y/N)',
       'Phone_Type', 'Call_Status', 'Contact_By_Phone_OK?_(Y/N)', 'City',
       'State/Province', 'Zip_Code', 'Bad_Address_Flag',
       'Contact_By_Mail_OK?_(Y/N)', 'Active_Flag', 'Deceased_Flag',
       'Primary_Connection_To_MS', 'Number_of_Team_Members'],
      dtype='object')

In [0]:
missing_team=ntactivity.Number_of_Team_Members.isnull()
ntactivity_missingteam=ntactivity[missing_team].copy()
ntactivity_missingteam #this will become our test data



In [0]:
ntactivity_missingteam['Number_of_Team_Members'].index

Int64Index([  3,  19,  22,  25,  54,  59,  70,  86,  91, 121, 130, 144, 174,
            194, 198, 211, 233, 235, 243, 285, 334, 349, 366, 383, 387, 394,
            402, 404, 408, 419, 436, 439, 475, 498, 582, 593, 595, 622, 635,
            637, 640, 650, 659, 673, 675, 688, 692, 702, 709, 713, 717, 727,
            744, 757, 807, 809, 814, 819, 845, 862, 874, 887, 893],
           dtype='int64')

In [0]:
having_team=~missing_team
having_team
ntactivity_havingteam=ntactivity[having_team].copy()
ntactivity_havingteam #this will become our test data

In [0]:
y=ntactivity_havingteam.Number_of_Team_Members
predictors=['Event_Category', 'Event_Type','Location', 'Corporate_Name', 'Revenue_Raised', 'City', 'State/Province',   'Primary_Connection_To_MS']
X=ntactivity_havingteam[predictors].copy()
X.shape


(747, 8)

In [0]:
X_enc=pd.get_dummies(X)
X_enc.shape

(747, 625)

In [0]:
X_missing=ntactivity_missingteam[predictors].copy()
X_missing_enc=pd.get_dummies(X_missing)
X_missing_enc.isnull().values.any()

False

In [0]:
col_test=X_missing_enc.columns
X_enc=X_enc.loc[:,col_test]
nullvalues=X_enc.isnull()
X_enc=X_enc.fillna(0.0)
X_enc.isnull().values.any()

False

In [0]:
train_X, val_X, train_y, val_y = train_test_split(X_enc, y,random_state = 0)

In [0]:
my_pipeline_xgbr = make_pipeline(XGBRegressor(n_estimators=1000,learning_rate=0.00999))
scores_xgbr = cross_val_score(my_pipeline_xgbr, train_X, train_y, scoring='neg_mean_squared_error')


New Mean Absolute Error 113.490189 XGBR


In [0]:
my_pipeline_gbr = make_pipeline(GradientBoostingRegressor())
scores_gbr = cross_val_score(my_pipeline_gbr, train_X, train_y, scoring='neg_mean_squared_error')

In [0]:
print('New Mean Absolute Error %2f XGBR' %(-1 * scores_xgbr.mean()))
print('New Mean Absolute Error %2f GBR' %(-1 * scores_gbr.mean()))


New Mean Absolute Error 126.749073 XGBR
New Mean Absolute Error 113.889859 GBR


In [0]:
my_model_GBR=GradientBoostingRegressor().fit(train_X,train_y)
val_predictions = my_model_GBR.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))
predicted_NOTM_GBR = my_model_GBR.predict(X_missing_enc)
print(predicted_NOTM_GBR.round().astype(int))

4.7235169658674
[ 3  3  3  3  3  3  3  8  3  3  3  3  3  3 10  3  3  3  3  3  3  3 14  3
  3  3  6  4  4  3  3  3  3  3  5  5  3  3  3  3  8  3  3  4  3  3  4  2
  3  3  3  3  3  3  3  5  3  3  3 29  3  4  4]


In [0]:
my_model_XGBR=XGBRegressor(n_estimators=1000,learning_rate=0.00999).fit(train_X,train_y)
val_predictions = my_model_XGBR.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))
predicted_NOTM_XGBR = my_model_XGBR.predict(X_missing_enc)
print(predicted_NOTM_XGBR.round().astype(int))

4.767150218474036
[ 3  3  6  3  3  5  3  7  3  3  3  3  3  3 10  3  3  3  3  3  3  3 13  3
  3  3  7  4  5  3  3  3  3  3  4  5  3  3  3  3  8  3  3  4  5  3  4  2
  3  3  3  3  6  3  3  3  3  4  3 28  3  3  6]


Using prediction from XGBR

In [0]:
predicted_NOTM=predicted_NOTM_XGBR.round().astype(int)
X_missing_enc['Number_of_Team_Members']=predicted_NOTM
notm_values=X_missing_enc['Number_of_Team_Members'].index
ntactivity.loc[notm_values,['Number_of_Team_Members']]=X_missing_enc['Number_of_Team_Members']

In [0]:
ntactivity.Number_of_Team_Members

In [0]:
ntactivity.isnull().values.any()

False

In [0]:
from google.colab import files

ntactivity.to_csv('ntactivity_clean.csv')

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'ntactivity_clean.csv'})
uploaded.SetContentFile('ntactivity_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1RgDsyUlY1Nf0wAw9xHG6uqCoqU5ACzu-


#Data Cleaning for donationsbike

In [0]:
donationsbike2017.isnull().sum().plot(kind='bar')

In [0]:
donationsbike2017=donationsbike2017.dropna(subset = ['Event_Date'])
donationsbike2017=donationsbike2017.fillna({'Gift_Payment_Method':'Credit Card','Offline_Status':'Not_Applicable','Donor_Member_ID':0,'Donor_ZIP':999999,'Participant_Contact_ID':0,'Donor_Connection_to_MS':'Other','Participant_Member_ID':0,'Fiscal_Year':0,'Donor_ConsID'	:0,'Donor_Opt-out_Date':'01/01/2000 00:00','Registration_Date':'01/01/2000 00:00','Team_Creation_Date':'01/01/2000 00:00'})
donationsbike2017=donationsbike2017.fillna('Unknown')


For the above row no the gift has been made by participant, payment is online and type is Credit card hence our intution is that the gift payment is Credit Card

From our understanding, offline status reflects if the payment has been made offline and confirnemd. Since for credit cards the payments are not confirmed immediately we will but the offline status as 'Not Applicable'

Updating generic IDs as 0 since the IDs are machine generated.

Zipcodes have been updated as a generic number 999999

In [0]:
donationsbike2017[	'Security_Category_Name'	]=donationsbike2017[	'Security_Category_Name'	].astype('category')
donationsbike2017[	'Public_Event_Name'	]=donationsbike2017[	'Public_Event_Name'	].astype('category')
donationsbike2017[	'Campaign_Title'	]=donationsbike2017[	'Campaign_Title'	].astype('category')
donationsbike2017[	'Gift_Type'	]=donationsbike2017[	'Gift_Type'	].astype('category')
donationsbike2017[	'Offline_Status'	]=donationsbike2017[	'Offline_Status'	].astype('category')
donationsbike2017[	'Is_Registration?'	]=donationsbike2017[	'Is_Registration?'	].astype('category')
donationsbike2017[	'Donor_Affiliate_Code'	]=donationsbike2017[	'Donor_Affiliate_Code'	].astype('category')
donationsbike2017[	'Donor_Gender'	]=donationsbike2017[	'Donor_Gender'	].astype('category')
donationsbike2017[	'Donor_Accept_Email'	]=donationsbike2017[	'Donor_Accept_Email'	].astype('category')
donationsbike2017[	'Donor_Opt-out_Method'	]=donationsbike2017[	'Donor_Opt-out_Method'	].astype('category')
donationsbike2017[	'Donor_Email_Status'	]=donationsbike2017[	'Donor_Email_Status'	].astype('category')
donationsbike2017[	'Donor_City'	]=donationsbike2017[	'Donor_City'	].astype('category')
donationsbike2017[	'Donor_State'	]=donationsbike2017[	'Donor_State'	].astype('category')
donationsbike2017[	'Donor_County'	]=donationsbike2017[	'Donor_County'	].astype('category')
donationsbike2017[	'Donor_Employer'	]=donationsbike2017[	'Donor_Employer'	].astype('category')
donationsbike2017[	'Donor_Connection_to_MS'	]=donationsbike2017[	'Donor_Connection_to_MS'	].astype('category')
donationsbike2017[	'Participation_Type_Name'	]=donationsbike2017[	'Participation_Type_Name'	].astype('category')
donationsbike2017[	'Registration_Active_Status'	]=donationsbike2017[	'Registration_Active_Status'	].astype('category')
donationsbike2017[	'Is_Prior_Participant'	]=donationsbike2017[	'Is_Prior_Participant'	].astype('category')
donationsbike2017[	'Is_Team_Captain'	]=donationsbike2017[	'Is_Team_Captain'	].astype('category')
donationsbike2017[	'Team_Name'	]=donationsbike2017[	'Team_Name'	].astype('category')
donationsbike2017[	'Team'	]=donationsbike2017[	'Team'	].astype('category')
donationsbike2017[	'Proxy_Type'	]=donationsbike2017[	'Proxy_Type'	].astype('category')
donationsbike2017[	'User_Confirmation_Code'	]=donationsbike2017[	'User_Confirmation_Code'	].astype('category')
donationsbike2017[	'Source_Code_Text'	]=donationsbike2017[	'Source_Code_Text'	].astype('category')
donationsbike2017[	'Sub_Source_Code_Text'	]=donationsbike2017[	'Sub_Source_Code_Text'	].astype('category')
donationsbike2017[	'Source_Code_Type'	]=donationsbike2017[	'Source_Code_Type'	].astype('category')
donationsbike2017[	'Batch_ID'	]=donationsbike2017[	'Batch_ID'	].astype('category')

In [0]:
donationsbike2017[	'Participant_Contact_ID'	]=donationsbike2017[	'Participant_Contact_ID'	].astype('int')

donationsbike2017['Additional_Gift_Amount($)']=pd.to_numeric(donationsbike2017['Additional_Gift_Amount($)'], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = ['Additional_Gift_Amount($)'])

donationsbike2017[	'Event_ID'	]=pd.to_numeric(donationsbike2017[	'Event_ID'	], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = [	'Event_ID'	])

donationsbike2017[	'Donor_Member_ID'	]=pd.to_numeric(donationsbike2017[	'Donor_Member_ID'	], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = [	'Donor_Member_ID'	])

donationsbike2017[	    'Campaign_ID'	]=pd.to_numeric(donationsbike2017[	    'Campaign_ID'	], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = [	    'Campaign_ID'	])

donationsbike2017[	'Donor_ConsID'	]=pd.to_numeric(donationsbike2017[	'Donor_ConsID'	], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = [	'Donor_ConsID'	])

donationsbike2017[	'Donor_ZIP'	]=pd.to_numeric(donationsbike2017[	'Donor_ZIP'	], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = [	'Donor_ZIP'	])

donationsbike2017[	'Participant_Member_ID'	]=pd.to_numeric(donationsbike2017[	'Participant_Member_ID'	], errors='coerce')
donationsbike2017=donationsbike2017.dropna(subset = [	'Participant_Member_ID'	])

donationsbike2017[	'Gift_Amount($)'	]=donationsbike2017[	'Gift_Amount($)'	].astype('float')
donationsbike2017[	'Participant_Goal($)']=	donationsbike2017[	'Participant_Goal($)'	].astype('float')
donationsbike2017[	'Additional_Gift_Amount($)']=	donationsbike2017[	'Additional_Gift_Amount($)'	].astype('float')
donationsbike2017[	'Campaign_ID'	]=donationsbike2017[	'Campaign_ID'	].astype('int')
donationsbike2017[	'Donor_Member_ID'	]=donationsbike2017[	'Donor_Member_ID'	].astype('int')
donationsbike2017[	'Donor_ZIP'	]=donationsbike2017[	'Donor_ZIP'	].astype('int')
donationsbike2017[	'Participant_Member_ID'	]=donationsbike2017[	'Participant_Member_ID'	].astype('int')
donationsbike2017[	'Donor_ConsID'	]=donationsbike2017[	'Donor_ConsID'	].astype('int')

In [89]:
filterbool=donationsbike2017['Team_Creation_Date'].apply(lambda x: len(x)<3)
donationsbike2017[filterbool]

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
9210,CAL Bike Events,28217,Bike MS: Coastal Challenge 2017,2017,3/25/2017 15:06,CAL 2017 Bike MS,44974,50.0,2017-02-22 18:27:00,online,...,50,50,50.0,TeamRaiser Participant,1638-44974-1-17910659-18882098,Unknown,Unknown,Unknown,Unknown,Unknown


In [0]:
donationsbike2017.drop(9210,inplace=True)

In [94]:
donationsbike2017[donationsbike2017['Event_Date']=='3/25+2017 15:06']

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
7740,CAL Bike Events,28217,Bike MS: Coastal Challenge 2017,2017,3/25+2017 15:06,CAL 2017 Bike MS,44974,200.0,2017-02-07 21:59:00,online,...,2016-08-29 14:12:00,200,200.0,200,TeamRaiser Participant,1638-44974-1-17867103-18834471,Unknown,Unknown,Unknown,Unknown


In [0]:
donationsbike2017.loc[7740,['Event_Date']]='3/25/2017 15:06'

In [0]:
donationsbike2017[	'Event_Date'	]=pd.to_datetime(donationsbike2017[	'Event_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2017[	'Date_Recorded'	]=pd.to_datetime(donationsbike2017[	'Date_Recorded'	],format="%m/%d/%Y %H:%M")
donationsbike2017[	'Donor_Opt-out_Date'	]=pd.to_datetime(donationsbike2017[	'Donor_Opt-out_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2017[	'Registration_Date'	]=pd.to_datetime(donationsbike2017[	'Registration_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2017[	'Team_Creation_Date'	]=pd.to_datetime(donationsbike2017[	'Team_Creation_Date'	],format="%m/%d/%Y %H:%M")


Since Additional_Gift_Columns has some wrong data records and they are just 6, it is best to drop them

Same concept is applied to other columns too

In [0]:
from google.colab import files
donationsbike2017.to_csv('donationsbike2017_clean.csv')


In [98]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'donationsbike2017_clean.csv'})
uploaded.SetContentFile('donationsbike2017_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1zK-8vfwC7J9XRJX0VjFuTdqrVcrdMImD


                        ---------------------------------- 2017 finished -------------------------------------------------------


In [0]:
donationsbike2016=donationsbike2016.dropna(subset = ['Event_Date'])
donationsbike2016=donationsbike2016.fillna({'Gift_Payment_Method':'Credit Card','Offline_Status':'Not_Applicable','Donor_Member_ID':0,'Donor_ZIP':999999,'Participant_Contact_ID':0,'Donor_Connection_to_MS':'Other','Participant_Member_ID':0,'Fiscal_Year':0,'Donor_ConsID'	:0,'Donor_Opt-out_Date':'01/01/2000 00:00','Registration_Date':'01/01/2000 00:00','Team_Creation_Date':'01/01/2000 00:00'})
donationsbike2016=donationsbike2016.fillna('Unknown')

In [0]:
donationsbike2016[	'Security_Category_Name'	]=donationsbike2016[	'Security_Category_Name'	].astype('category')
donationsbike2016[	'Public_Event_Name'	]=donationsbike2016[	'Public_Event_Name'	].astype('category')
donationsbike2016[	'Campaign_Title'	]=donationsbike2016[	'Campaign_Title'	].astype('category')
donationsbike2016[	'Gift_Type'	]=donationsbike2016[	'Gift_Type'	].astype('category')
donationsbike2016[	'Offline_Status'	]=donationsbike2016[	'Offline_Status'	].astype('category')
donationsbike2016[	'Is_Registration?'	]=donationsbike2016[	'Is_Registration?'	].astype('category')
donationsbike2016[	'Donor_Affiliate_Code'	]=donationsbike2016[	'Donor_Affiliate_Code'	].astype('category')
donationsbike2016[	'Donor_Gender'	]=donationsbike2016[	'Donor_Gender'	].astype('category')
donationsbike2016[	'Donor_Accept_Email'	]=donationsbike2016[	'Donor_Accept_Email'	].astype('category')
donationsbike2016[	'Donor_Opt-out_Method'	]=donationsbike2016[	'Donor_Opt-out_Method'	].astype('category')
donationsbike2016[	'Donor_Email_Status'	]=donationsbike2016[	'Donor_Email_Status'	].astype('category')
donationsbike2016[	'Donor_City'	]=donationsbike2016[	'Donor_City'	].astype('category')
donationsbike2016[	'Donor_State'	]=donationsbike2016[	'Donor_State'	].astype('category')
donationsbike2016[	'Donor_County'	]=donationsbike2016[	'Donor_County'	].astype('category')
donationsbike2016[	'Donor_Employer'	]=donationsbike2016[	'Donor_Employer'	].astype('category')
donationsbike2016[	'Donor_Connection_to_MS'	]=donationsbike2016[	'Donor_Connection_to_MS'	].astype('category')
donationsbike2016[	'Participation_Type_Name'	]=donationsbike2016[	'Participation_Type_Name'	].astype('category')
donationsbike2016[	'Registration_Active_Status'	]=donationsbike2016[	'Registration_Active_Status'	].astype('category')
donationsbike2016[	'Is_Prior_Participant'	]=donationsbike2016[	'Is_Prior_Participant'	].astype('category')
donationsbike2016[	'Is_Team_Captain'	]=donationsbike2016[	'Is_Team_Captain'	].astype('category')
donationsbike2016[	'Team_Name'	]=donationsbike2016[	'Team_Name'	].astype('category')
donationsbike2016[	'Team'	]=donationsbike2016[	'Team'	].astype('category')
donationsbike2016[	'Proxy_Type'	]=donationsbike2016[	'Proxy_Type'	].astype('category')
donationsbike2016[	'User_Confirmation_Code'	]=donationsbike2016[	'User_Confirmation_Code'	].astype('category')
donationsbike2016[	'Source_Code_Text'	]=donationsbike2016[	'Source_Code_Text'	].astype('category')
donationsbike2016[	'Sub_Source_Code_Text'	]=donationsbike2016[	'Sub_Source_Code_Text'	].astype('category')
donationsbike2016[	'Source_Code_Type'	]=donationsbike2016[	'Source_Code_Type'	].astype('category')
donationsbike2016[	'Batch_ID'	]=donationsbike2016[	'Batch_ID'	].astype('category')

In [0]:

donationsbike2016[	'Participant_Contact_ID'	]=donationsbike2016[	'Participant_Contact_ID'	].astype('int')

donationsbike2016['Additional_Gift_Amount($)']=pd.to_numeric(donationsbike2016['Additional_Gift_Amount($)'], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = ['Additional_Gift_Amount($)'])

donationsbike2016[	'Event_ID'	]=pd.to_numeric(donationsbike2016[	'Event_ID'	], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = [	'Event_ID'	])

donationsbike2016[	'Donor_Member_ID'	]=pd.to_numeric(donationsbike2016[	'Donor_Member_ID'	], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = [	'Donor_Member_ID'	])

donationsbike2016[	    'Campaign_ID'	]=pd.to_numeric(donationsbike2016[	    'Campaign_ID'	], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = [	    'Campaign_ID'	])

donationsbike2016[	'Donor_ConsID'	]=pd.to_numeric(donationsbike2016[	'Donor_ConsID'	], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = [	'Donor_ConsID'	])

donationsbike2016[	'Donor_ZIP'	]=pd.to_numeric(donationsbike2016[	'Donor_ZIP'	], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = [	'Donor_ZIP'	])

donationsbike2016[	'Participant_Member_ID'	]=pd.to_numeric(donationsbike2016[	'Participant_Member_ID'	], errors='coerce')
donationsbike2016=donationsbike2016.dropna(subset = [	'Participant_Member_ID'	])

donationsbike2016[	'Gift_Amount($)'	]=donationsbike2016[	'Gift_Amount($)'	].astype('float')
donationsbike2016[	'Participant_Goal($)']=	donationsbike2016[	'Participant_Goal($)'	].astype('float')
donationsbike2016[	'Additional_Gift_Amount($)']=	donationsbike2016[	'Additional_Gift_Amount($)'	].astype('float')
donationsbike2016[	'Campaign_ID'	]=donationsbike2016[	'Campaign_ID'	].astype('int')
donationsbike2016[	'Donor_Member_ID'	]=donationsbike2016[	'Donor_Member_ID'	].astype('int')
donationsbike2016[	'Donor_ZIP'	]=donationsbike2016[	'Donor_ZIP'	].astype('int')
donationsbike2016[	'Participant_Member_ID'	]=donationsbike2016[	'Participant_Member_ID'	].astype('int')
donationsbike2016[	'Donor_ConsID'	]=donationsbike2016[	'Donor_ConsID'	].astype('int')

In [66]:
donationsbike2016[donationsbike2016['Event_Date']=='4/16+2016 13:18']

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
484986,TXH Bike Events,27003,2016 BP MS 150,2016,4/16+2016 13:18,TXH 2016 Bike MS: BP MS 150,44120,75.0,12/30/2015 12:23,online,...,9/15/2015 12:41,175.0,175.0,175,TeamRaiser Participant,1638-44120-20-16239491-16988359,Unknown,Unknown,Unknown,Unknown


In [0]:
donationsbike2016.loc[484986,['Event_Date']]='4/16/2016 13:18'

In [75]:
#'(100'
donationsbike2016[donationsbike2016['Team_Creation_Date']=='(100']

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
553994,TXH Bike Events,27003,2016 BP MS 150,2016,2016-04-16 13:18:00,TXH 2016 Bike MS: BP MS 150,44120,100.0,2016-04-13 19:21:00,online,...,(100,100.0,100.0,TeamRaiser Participant,1638-44120-1-16769191-17589979,Unknown,Unknown,Unknown,Unknown,Unknown


In [0]:
donationsbike2016.drop(553994,inplace=True)

In [0]:

#donationsbike2016[	'Event_Date'	]=pd.to_datetime(donationsbike2016[	'Event_Date'	],format="%m/%d/%Y %H:%M")
#donationsbike2016[	'Date_Recorded'	]=pd.to_datetime(donationsbike2016[	'Date_Recorded'	],format="%m/%d/%Y %H:%M")
#donationsbike2016[	'Donor_Opt-out_Date'	]=pd.to_datetime(donationsbike2016[	'Donor_Opt-out_Date'	],format="%m/%d/%Y %H:%M")
#donationsbike2016[	'Registration_Date'	]=pd.to_datetime(donationsbike2016[	'Registration_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2016[	'Team_Creation_Date'	]=pd.to_datetime(donationsbike2016[	'Team_Creation_Date'	],format="%m/%d/%Y %H:%M")


In [0]:
from google.colab import files
donationsbike2016.to_csv('donationsbike2016_clean.csv')

In [79]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'donationsbike2016_clean.csv'})
uploaded.SetContentFile('donationsbike2016_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1PprsMwobIyGayurdT0knujWRfGghpOZ4


                                  ------------------------------------- 2016 finished ------------------------------------------

In [0]:
donationsbike2015=donationsbike2015.dropna(subset = ['Event_Date'])
donationsbike2015=donationsbike2015.fillna({'Gift_Payment_Method':'Credit Card','Offline_Status':'Not_Applicable','Donor_Member_ID':0,'Donor_ZIP':999999,'Participant_Contact_ID':0,'Donor_Connection_to_MS':'Other','Participant_Member_ID':0,'Fiscal_Year':0,'Donor_ConsID'	:0,'Donor_Opt-out_Date':'01/01/2000 00:00','Registration_Date':'01/01/2000 00:00','Team_Creation_Date':'01/01/2000 00:00'})
donationsbike2015=donationsbike2015.fillna('Unknown')


In [0]:
donationsbike2015[	'Security_Category_Name'	]=donationsbike2015[	'Security_Category_Name'	].astype('category')
donationsbike2015[	'Public_Event_Name'	]=donationsbike2015[	'Public_Event_Name'	].astype('category')
donationsbike2015[	'Campaign_Title'	]=donationsbike2015[	'Campaign_Title'	].astype('category')
donationsbike2015[	'Gift_Type'	]=donationsbike2015[	'Gift_Type'	].astype('category')
donationsbike2015[	'Offline_Status'	]=donationsbike2015[	'Offline_Status'	].astype('category')
donationsbike2015[	'Is_Registration?'	]=donationsbike2015[	'Is_Registration?'	].astype('category')
donationsbike2015[	'Donor_Affiliate_Code'	]=donationsbike2015[	'Donor_Affiliate_Code'	].astype('category')
donationsbike2015[	'Donor_Gender'	]=donationsbike2015[	'Donor_Gender'	].astype('category')
donationsbike2015[	'Donor_Accept_Email'	]=donationsbike2015[	'Donor_Accept_Email'	].astype('category')
donationsbike2015[	'Donor_Opt-out_Method'	]=donationsbike2015[	'Donor_Opt-out_Method'	].astype('category')
donationsbike2015[	'Donor_Email_Status'	]=donationsbike2015[	'Donor_Email_Status'	].astype('category')
donationsbike2015[	'Donor_City'	]=donationsbike2015[	'Donor_City'	].astype('category')
donationsbike2015[	'Donor_State'	]=donationsbike2015[	'Donor_State'	].astype('category')
donationsbike2015[	'Donor_County'	]=donationsbike2015[	'Donor_County'	].astype('category')
donationsbike2015[	'Donor_Employer'	]=donationsbike2015[	'Donor_Employer'	].astype('category')
donationsbike2015[	'Donor_Connection_to_MS'	]=donationsbike2015[	'Donor_Connection_to_MS'	].astype('category')
donationsbike2015[	'Participation_Type_Name'	]=donationsbike2015[	'Participation_Type_Name'	].astype('category')
donationsbike2015[	'Registration_Active_Status'	]=donationsbike2015[	'Registration_Active_Status'	].astype('category')
donationsbike2015[	'Is_Prior_Participant'	]=donationsbike2015[	'Is_Prior_Participant'	].astype('category')
donationsbike2015[	'Is_Team_Captain'	]=donationsbike2015[	'Is_Team_Captain'	].astype('category')
donationsbike2015[	'Team_Name'	]=donationsbike2015[	'Team_Name'	].astype('category')
donationsbike2015[	'Team'	]=donationsbike2015[	'Team'	].astype('category')
donationsbike2015[	'Proxy_Type'	]=donationsbike2015[	'Proxy_Type'	].astype('category')
donationsbike2015[	'User_Confirmation_Code'	]=donationsbike2015[	'User_Confirmation_Code'	].astype('category')
donationsbike2015[	'Source_Code_Text'	]=donationsbike2015[	'Source_Code_Text'	].astype('category')
donationsbike2015[	'Sub_Source_Code_Text'	]=donationsbike2015[	'Sub_Source_Code_Text'	].astype('category')
donationsbike2015[	'Source_Code_Type'	]=donationsbike2015[	'Source_Code_Type'	].astype('category')
donationsbike2015[	'Batch_ID'	]=donationsbike2015[	'Batch_ID'	].astype('category')

In [0]:

donationsbike2015[	'Participant_Contact_ID'	]=donationsbike2015[	'Participant_Contact_ID'	].astype('int')

donationsbike2015['Additional_Gift_Amount($)']=pd.to_numeric(donationsbike2015['Additional_Gift_Amount($)'], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = ['Additional_Gift_Amount($)'])

donationsbike2015[	'Event_ID'	]=pd.to_numeric(donationsbike2015[	'Event_ID'	], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = [	'Event_ID'	])

donationsbike2015[	'Donor_Member_ID'	]=pd.to_numeric(donationsbike2015[	'Donor_Member_ID'	], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = [	'Donor_Member_ID'	])

donationsbike2015[	    'Campaign_ID'	]=pd.to_numeric(donationsbike2015[	    'Campaign_ID'	], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = [	    'Campaign_ID'	])

donationsbike2015[	'Donor_ConsID'	]=pd.to_numeric(donationsbike2015[	'Donor_ConsID'	], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = [	'Donor_ConsID'	])

donationsbike2015[	'Donor_ZIP'	]=pd.to_numeric(donationsbike2015[	'Donor_ZIP'	], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = [	'Donor_ZIP'	])

donationsbike2015[	'Participant_Member_ID'	]=pd.to_numeric(donationsbike2015[	'Participant_Member_ID'	], errors='coerce')
donationsbike2015=donationsbike2015.dropna(subset = [	'Participant_Member_ID'	])

donationsbike2015[	'Gift_Amount($)'	]=donationsbike2015[	'Gift_Amount($)'	].astype('float')
donationsbike2015[	'Participant_Goal($)']=	donationsbike2015[	'Participant_Goal($)'	].astype('float')
donationsbike2015[	'Additional_Gift_Amount($)']=	donationsbike2015[	'Additional_Gift_Amount($)'	].astype('float')
donationsbike2015[	'Campaign_ID'	]=donationsbike2015[	'Campaign_ID'	].astype('int')
donationsbike2015[	'Donor_Member_ID'	]=donationsbike2015[	'Donor_Member_ID'	].astype('int')
donationsbike2015[	'Donor_ZIP'	]=donationsbike2015[	'Donor_ZIP'	].astype('int')
donationsbike2015[	'Participant_Member_ID'	]=donationsbike2015[	'Participant_Member_ID'	].astype('int')
donationsbike2015[	'Donor_ConsID'	]=donationsbike2015[	'Donor_ConsID'	].astype('int')

In [31]:
donationsbike2015[donationsbike2015['Event_Date']=='8/1+2015 12:21']

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
775038,WIG Bike Events,24838,Bike MS: TOYOTA Best Dam Bike Tour 2015,2015,8/1+2015 12:21,WIG Bike Donation Campaign,35582,50.0,9/1/2015 0:00,offline,...,8/6/2014 10:11,50.0,50,50,TeamRaiser Participant,1638-35582-1-15958851-16677134,Unknown,Unknown,Unknown,WIG09022015KAKDP16545S


Since this is not a valid date and 1+ can be anything, we will drop this row

In [0]:
donationsbike2015.drop(775038,inplace=True)

In [52]:

donationsbike2015[donationsbike2015[	'Registration_Date'	]=='3/17+2015 15:04']

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
437804,PAE Bike Events,25223,Bike MS: City to Shore Ride 2015,2015,2015-09-26 10:47:00,PAE Bike MS 2015 Donation Campaign,41715,50.0,2015-09-16 20:02:00,online,...,1/20/2015 7:01,50.0,50,50,TeamRaiser Participant,1638-41715-1-16014136-16737262,Unknown,Unknown,Unknown,Unknown


In [35]:
donationsbike2015[donationsbike2015[	'Date_Recorded'	]=='8/20+2015 19:17']

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
422012,OHG Bike Events,24907,Ohio - Bike MS: Venture the Valley 2015,2015,2015-08-22 13:10:00,OHG 2015 Bike Donation Campaign,41455,35.0,8/20+2015 19:17,online,...,3/13/2015 15:10,35.0,35,35,TeamRaiser Participant,1638-41455-1-15877004-16587399,Unknown,Unknown,Unknown,Unknown


In [0]:
#invalid date, it cannot be anything except 8/20/2015
donationsbike2015.loc[422012,['Date_Recorded']]='8/20/2015 19:17'
#similiarly for other dates
donationsbike2015.loc[450031,['Date_Recorded']]='9/22/2015 22:06'
donationsbike2015.loc[294556	,[	'Registration_Date'	]]='4/14/2015 16:44'
donationsbike2015.loc[437804	,[	'Registration_Date'	]]='3/17/2015 15:04'

In [0]:
donationsbike2015[	'Event_Date'	]=pd.to_datetime(donationsbike2015[	'Event_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2015[	'Date_Recorded'	]=pd.to_datetime(donationsbike2015[	'Date_Recorded'	],format="%m/%d/%Y %H:%M")
donationsbike2015[	'Donor_Opt-out_Date'	]=pd.to_datetime(donationsbike2015[	'Donor_Opt-out_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2015[	'Registration_Date'	]=pd.to_datetime(donationsbike2015[	'Registration_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2015[	'Team_Creation_Date'	]=pd.to_datetime(donationsbike2015[	'Team_Creation_Date'	],format="%m/%d/%Y %H:%M")

In [0]:
from google.colab import files
donationsbike2015.to_csv('donationsbike2015_clean.csv')

In [59]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'donationsbike2015_clean.csv'})
uploaded.SetContentFile('donationsbike2015_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 17wi-qILuRHa3czHHBtvO0Hx6m1iD1r6V


                                  ------------------------------------- 2015 finished ------------------------------------------

In [0]:

donationsbike2014=donationsbike2014.dropna(subset = ['Event_Date'])
donationsbike2014=donationsbike2014.fillna({'Gift_Payment_Method':'Credit Card','Offline_Status':'Not_Applicable','Donor_Member_ID':0,'Donor_ZIP':999999,'Participant_Contact_ID':0,'Donor_Connection_to_MS':'Other','Participant_Member_ID':0,'Fiscal_Year':0,'Donor_ConsID'	:0,'Donor_Opt-out_Date':'01/01/2000 00:00','Registration_Date':'01/01/2000 00:00','Team_Creation_Date':'01/01/2000 00:00'})
donationsbike2014=donationsbike2014.fillna('Unknown')




In [0]:

donationsbike2014[	'Security_Category_Name'	]=donationsbike2014[	'Security_Category_Name'	].astype('category')
donationsbike2014[	'Public_Event_Name'	]=donationsbike2014[	'Public_Event_Name'	].astype('category')
donationsbike2014[	'Campaign_Title'	]=donationsbike2014[	'Campaign_Title'	].astype('category')
donationsbike2014[	'Gift_Type'	]=donationsbike2014[	'Gift_Type'	].astype('category')
donationsbike2014[	'Offline_Status'	]=donationsbike2014[	'Offline_Status'	].astype('category')
donationsbike2014[	'Is_Registration?'	]=donationsbike2014[	'Is_Registration?'	].astype('category')
donationsbike2014[	'Donor_Affiliate_Code'	]=donationsbike2014[	'Donor_Affiliate_Code'	].astype('category')
donationsbike2014[	'Donor_Gender'	]=donationsbike2014[	'Donor_Gender'	].astype('category')
donationsbike2014[	'Donor_Accept_Email'	]=donationsbike2014[	'Donor_Accept_Email'	].astype('category')
donationsbike2014[	'Donor_Opt-out_Method'	]=donationsbike2014[	'Donor_Opt-out_Method'	].astype('category')
donationsbike2014[	'Donor_Email_Status'	]=donationsbike2014[	'Donor_Email_Status'	].astype('category')
donationsbike2014[	'Donor_City'	]=donationsbike2014[	'Donor_City'	].astype('category')
donationsbike2014[	'Donor_State'	]=donationsbike2014[	'Donor_State'	].astype('category')
donationsbike2014[	'Donor_County'	]=donationsbike2014[	'Donor_County'	].astype('category')
donationsbike2014[	'Donor_Employer'	]=donationsbike2014[	'Donor_Employer'	].astype('category')
donationsbike2014[	'Donor_Connection_to_MS'	]=donationsbike2014[	'Donor_Connection_to_MS'	].astype('category')
donationsbike2014[	'Participation_Type_Name'	]=donationsbike2014[	'Participation_Type_Name'	].astype('category')
donationsbike2014[	'Registration_Active_Status'	]=donationsbike2014[	'Registration_Active_Status'	].astype('category')
donationsbike2014[	'Is_Prior_Participant'	]=donationsbike2014[	'Is_Prior_Participant'	].astype('category')
donationsbike2014[	'Is_Team_Captain'	]=donationsbike2014[	'Is_Team_Captain'	].astype('category')
donationsbike2014[	'Team_Name'	]=donationsbike2014[	'Team_Name'	].astype('category')
donationsbike2014[	'Team'	]=donationsbike2014[	'Team'	].astype('category')
donationsbike2014[	'Proxy_Type'	]=donationsbike2014[	'Proxy_Type'	].astype('category')
donationsbike2014[	'User_Confirmation_Code'	]=donationsbike2014[	'User_Confirmation_Code'	].astype('category')
donationsbike2014[	'Source_Code_Text'	]=donationsbike2014[	'Source_Code_Text'	].astype('category')
donationsbike2014[	'Sub_Source_Code_Text'	]=donationsbike2014[	'Sub_Source_Code_Text'	].astype('category')
donationsbike2014[	'Source_Code_Type'	]=donationsbike2014[	'Source_Code_Type'	].astype('category')
donationsbike2014[	'Batch_ID'	]=donationsbike2014[	'Batch_ID'	].astype('category')

In [0]:


donationsbike2014[	'Participant_Contact_ID'	]=donationsbike2014[	'Participant_Contact_ID'	].astype('int')

donationsbike2014['Additional_Gift_Amount($)']=pd.to_numeric(donationsbike2014['Additional_Gift_Amount($)'], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = ['Additional_Gift_Amount($)'])


donationsbike2014[	'Event_ID'	]=pd.to_numeric(donationsbike2014[	'Event_ID'	], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = [	'Event_ID'	])

donationsbike2014[	'Donor_Member_ID'	]=pd.to_numeric(donationsbike2014[	'Donor_Member_ID'	], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = [	'Donor_Member_ID'	])


donationsbike2014[	    'Campaign_ID'	]=pd.to_numeric(donationsbike2014[	    'Campaign_ID'	], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = [	    'Campaign_ID'	])



donationsbike2014[	'Donor_ConsID'	]=pd.to_numeric(donationsbike2014[	'Donor_ConsID'	], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = [	'Donor_ConsID'	])



donationsbike2014[	'Donor_ZIP'	]=pd.to_numeric(donationsbike2014[	'Donor_ZIP'	], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = [	'Donor_ZIP'	])

donationsbike2014[	'Participant_Member_ID'	]=pd.to_numeric(donationsbike2014[	'Participant_Member_ID'	], errors='coerce')
donationsbike2014=donationsbike2014.dropna(subset = [	'Participant_Member_ID'	])








donationsbike2014[	'Gift_Amount($)'	]=donationsbike2014[	'Gift_Amount($)'	].astype('float')
donationsbike2014[	'Participant_Goal($)']=	donationsbike2014[	'Participant_Goal($)'	].astype('float')
donationsbike2014[	'Additional_Gift_Amount($)']=	donationsbike2014[	'Additional_Gift_Amount($)'	].astype('float')
donationsbike2014[	'Campaign_ID'	]=donationsbike2014[	'Campaign_ID'	].astype('int')
donationsbike2014[	'Donor_Member_ID'	]=donationsbike2014[	'Donor_Member_ID'	].astype('int')
donationsbike2014[	'Donor_ZIP'	]=donationsbike2014[	'Donor_ZIP'	].astype('int')
donationsbike2014[	'Participant_Member_ID'	]=donationsbike2014[	'Participant_Member_ID'	].astype('int')
donationsbike2014[	'Donor_ConsID'	]=donationsbike2014[	'Donor_ConsID'	].astype('int')

In [0]:
donationsbike2014[	'Event_Date'	]=pd.to_datetime(donationsbike2014[	'Event_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2014[	'Date_Recorded'	]=pd.to_datetime(donationsbike2014[	'Date_Recorded'	],format="%m/%d/%Y %H:%M")
donationsbike2014[	'Donor_Opt-out_Date'	]=pd.to_datetime(donationsbike2014[	'Donor_Opt-out_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2014[	'Registration_Date'	]=pd.to_datetime(donationsbike2014[	'Registration_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2014[	'Team_Creation_Date'	]=pd.to_datetime(donationsbike2014[	'Team_Creation_Date'	],format="%m/%d/%Y %H:%M")


In [0]:
from google.colab import files
donationsbike2014.to_csv('donationsbike2014_clean.csv')

In [0]:
# Create & upload a file.
uploaded = drive.CreateFile({'title': 'donationsbike2014_clean.csv'})
uploaded.SetContentFile('donationsbike2014_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1captSN7r1QRM4qDdIX4l_Kf-qQbO09r3


      ------------------------------------- 2014 finished ------------------------------------------

In [0]:
donationsbike2013=donationsbike2013.dropna(subset = ['Event_Date'])
donationsbike2013=donationsbike2013.fillna({'Gift_Payment_Method':'Credit Card','Offline_Status':'Not_Applicable','Donor_Member_ID':0,'Donor_ZIP':999999,'Participant_Contact_ID':0,'Donor_Connection_to_MS':'Other','Participant_Member_ID':0,'Fiscal_Year':0,'Donor_ConsID'	:0,'Donor_Opt-out_Date':'01/01/2000 00:00','Registration_Date':'01/01/2000 00:00','Team_Creation_Date':'01/01/2000 00:00'})
donationsbike2013=donationsbike2013.fillna('Unknown')

Date_Recorded    0/29/2013 21:42
Name: 292537, dtype: object
Since this is an invalid format and we cannot predict the actual date we will drop this

In [0]:
donationsbike2013.drop( 292537 ,inplace=True)

In [0]:
dindex=donationsbike2013.index

In [15]:
filterbool=donationsbike2013['Team_Creation_Date'].apply(lambda x: len(x)<3)
donationsbike2013[filterbool]

,Security_Category_Name,Event_ID,Public_Event_Name,Fiscal_Year,Event_Date,Campaign_Title,Campaign_ID,Gift_Amount($),Date_Recorded,Gift_Type,...,Team_Creation_Date,Original_Value_Transacted,Net_Transaction_Amount,Ledger_Transaction_Amount,Proxy_Type,User_Confirmation_Code,Source_Code_Type,Source_Code_Text,Sub_Source_Code_Text,Batch_ID
175502,ILD Bike Events,20718,ILD Bike MS: Tour de Farms 2013,2013,6/22/2013 11:50,ILD Bike MS 2013 Donation Campaign,36072,50.0,7/19/2013 14:29,offline,...,50,50.0,50,TeamRaiser Team,1638-36072-1-12281433-12721904,Unknown,Unknown,Unknown,ILD042913MB,Unknown
209938,KSG Bike Events,20078,Bike MS 2013 Ozarks Ride,2013,9/7/2013 11:56,KSG Bike Donation Campaign 2013,35132,50.0,9/5/2013 13:28(offline,Check,...,50,50.0,50,TeamRaiser Participant,1638-35132-1-12497403-12952191,6,intranet.nmss.org,/Pages/Home.aspx,KSG090513KL2,Unknown
321402,MNM Bike Events,19263,Minnesota - Bike MS: C.H. Robinson MS 150 Ride...,2013,6/7/2013 14:09,MNM 2013 Bike MS: MS 150 Ride,34312,20.0,1/24/2013 20:50,online,...,65,65.0,65,TeamRaiser Participant,1638-34312-20-11084041-11446454,Unknown,Unknown,Unknown,Unknown,Unknown
485629,OHA Bike Events,20976,Ohio - Bike MS: Pedal to the Point Ride 2013,2013,8/3/2013 15:27,OHA Bike Donation Campaign,35566,71.0,8/28/2013 15:07,offline,...,71,71.0,71,TeamRaiser Participant,1638-35566-1-12454353-12906856,Unknown,Unknown,Unknown,OHA082813-3617ER,Unknown
622886,TNS Bike Events,18277,"Tennessee, Franklin 2012 Bike to Jack and Back",2013,10/6/2012 14:14,TNS Bike to Jack and Back 2012,32787,50.0,2/14/2012 16:52,online,...,64,64.0,64,TeamRaiser Participant,1638-32787-20-9191167-9459366,Unknown,Unknown,Unknown,Unknown,Unknown
801593,TXH Bike Events,18129,Bike MS: Valero 2012 Alamo Ride to the River p...,2013,10/13/2012 14:53,"Texas, San Antonio - Bike MS: Valero 2012 Alam...",32633,25.0,10/17/2012 0:00,offline,...,25,25.0,25,TeamRaiser Participant,1638-32633-1-10962921-11314738,Unknown,Unknown,Unknown,TXH20121017-40546900-RR,Unknown


Since these dates are not valid and we cannot impute it , we will drop these rows

In [0]:
donationsbike2013.drop(donationsbike2013[filterbool].index,inplace=True)

In [0]:
donationsbike2013[	'Security_Category_Name'	]=donationsbike2013[	'Security_Category_Name'	].astype('category')
donationsbike2013[	'Public_Event_Name'	]=donationsbike2013[	'Public_Event_Name'	].astype('category')
donationsbike2013[	'Campaign_Title'	]=donationsbike2013[	'Campaign_Title'	].astype('category')
donationsbike2013[	'Gift_Type'	]=donationsbike2013[	'Gift_Type'	].astype('category')
donationsbike2013[	'Offline_Status'	]=donationsbike2013[	'Offline_Status'	].astype('category')
donationsbike2013[	'Is_Registration?'	]=donationsbike2013[	'Is_Registration?'	].astype('category')
donationsbike2013[	'Donor_Affiliate_Code'	]=donationsbike2013[	'Donor_Affiliate_Code'	].astype('category')
donationsbike2013[	'Donor_Gender'	]=donationsbike2013[	'Donor_Gender'	].astype('category')
donationsbike2013[	'Donor_Accept_Email'	]=donationsbike2013[	'Donor_Accept_Email'	].astype('category')
donationsbike2013[	'Donor_Opt-out_Method'	]=donationsbike2013[	'Donor_Opt-out_Method'	].astype('category')
donationsbike2013[	'Donor_Email_Status'	]=donationsbike2013[	'Donor_Email_Status'	].astype('category')
donationsbike2013[	'Donor_City'	]=donationsbike2013[	'Donor_City'	].astype('category')
donationsbike2013[	'Donor_State'	]=donationsbike2013[	'Donor_State'	].astype('category')
donationsbike2013[	'Donor_County'	]=donationsbike2013[	'Donor_County'	].astype('category')
donationsbike2013[	'Donor_Employer'	]=donationsbike2013[	'Donor_Employer'	].astype('category')
donationsbike2013[	'Donor_Connection_to_MS'	]=donationsbike2013[	'Donor_Connection_to_MS'	].astype('category')
donationsbike2013[	'Participation_Type_Name'	]=donationsbike2013[	'Participation_Type_Name'	].astype('category')
donationsbike2013[	'Registration_Active_Status'	]=donationsbike2013[	'Registration_Active_Status'	].astype('category')
donationsbike2013[	'Is_Prior_Participant'	]=donationsbike2013[	'Is_Prior_Participant'	].astype('category')
donationsbike2013[	'Is_Team_Captain'	]=donationsbike2013[	'Is_Team_Captain'	].astype('category')
donationsbike2013[	'Team_Name'	]=donationsbike2013[	'Team_Name'	].astype('category')
donationsbike2013[	'Team'	]=donationsbike2013[	'Team'	].astype('category')
donationsbike2013[	'Proxy_Type'	]=donationsbike2013[	'Proxy_Type'	].astype('category')
donationsbike2013[	'User_Confirmation_Code'	]=donationsbike2013[	'User_Confirmation_Code'	].astype('category')
donationsbike2013[	'Source_Code_Text'	]=donationsbike2013[	'Source_Code_Text'	].astype('category')
donationsbike2013[	'Sub_Source_Code_Text'	]=donationsbike2013[	'Sub_Source_Code_Text'	].astype('category')
donationsbike2013[	'Source_Code_Type'	]=donationsbike2013[	'Source_Code_Type'	].astype('category')
donationsbike2013[	'Batch_ID'	]=donationsbike2013[	'Batch_ID'	].astype('category')

In [0]:
donationsbike2013[	'Participant_Contact_ID'	]=donationsbike2013[	'Participant_Contact_ID'	].astype('int')

donationsbike2013['Additional_Gift_Amount($)']=pd.to_numeric(donationsbike2013['Additional_Gift_Amount($)'], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = ['Additional_Gift_Amount($)'])

donationsbike2013[	'Event_ID'	]=pd.to_numeric(donationsbike2013[	'Event_ID'	], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = [	'Event_ID'	])

donationsbike2013[	'Donor_Member_ID'	]=pd.to_numeric(donationsbike2013[	'Donor_Member_ID'	], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = [	'Donor_Member_ID'	])

donationsbike2013[	    'Campaign_ID'	]=pd.to_numeric(donationsbike2013[	    'Campaign_ID'	], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = [	    'Campaign_ID'	])

donationsbike2013[	'Donor_ConsID'	]=pd.to_numeric(donationsbike2013[	'Donor_ConsID'	], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = [	'Donor_ConsID'	])

donationsbike2013[	'Donor_ZIP'	]=pd.to_numeric(donationsbike2013[	'Donor_ZIP'	], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = [	'Donor_ZIP'	])

donationsbike2013[	'Participant_Member_ID'	]=pd.to_numeric(donationsbike2013[	'Participant_Member_ID'	], errors='coerce')
donationsbike2013=donationsbike2013.dropna(subset = [	'Participant_Member_ID'	])

donationsbike2013[	'Gift_Amount($)'	]=donationsbike2013[	'Gift_Amount($)'	].astype('float')
donationsbike2013[	'Participant_Goal($)']=	donationsbike2013[	'Participant_Goal($)'	].astype('float')
donationsbike2013[	'Additional_Gift_Amount($)']=	donationsbike2013[	'Additional_Gift_Amount($)'	].astype('float')
donationsbike2013[	'Campaign_ID'	]=donationsbike2013[	'Campaign_ID'	].astype('int')
donationsbike2013[	'Donor_Member_ID'	]=donationsbike2013[	'Donor_Member_ID'	].astype('int')
donationsbike2013[	'Donor_ZIP'	]=donationsbike2013[	'Donor_ZIP'	].astype('int')
donationsbike2013[	'Participant_Member_ID'	]=donationsbike2013[	'Participant_Member_ID'	].astype('int')
donationsbike2013[	'Donor_ConsID'	]=donationsbike2013[	'Donor_ConsID'	].astype('int')


In [0]:
donationsbike2013[	'Event_Date'	]=pd.to_datetime(donationsbike2013[	'Event_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2013[	'Date_Recorded'	]=pd.to_datetime(donationsbike2013[	'Date_Recorded'	],format="%m/%d/%Y %H:%M")
donationsbike2013[	'Donor_Opt-out_Date'	]=pd.to_datetime(donationsbike2013[	'Donor_Opt-out_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2013[	'Registration_Date'	]=pd.to_datetime(donationsbike2013[	'Registration_Date'	],format="%m/%d/%Y %H:%M")
donationsbike2013[	'Team_Creation_Date'	]=pd.to_datetime(donationsbike2013[	'Team_Creation_Date'	],format="%m/%d/%Y %H:%M")

In [0]:
donationsbike2013.dtypes

In [0]:
from google.colab import files
donationsbike2013.to_csv('donationsbike2013_clean.csv')

In [24]:
#create & upload a file.
uploaded = drive.CreateFile({'title': 'donationsbike2013_clean.csv'})
uploaded.SetContentFile('donationsbike2013_clean.csv')
uploaded.Upload()
print('Uploaded file with ID {}'.format(uploaded.get('id')))

Uploaded file with ID 1YD_0Ujg8b2zFozA-O4FfMiBreJT1c2Wu


#**Predicting Number of teams **

In [0]:
teamcount=ntactivity.loc[:,['Number_of_Team_Members']]
teamcount

In [0]:
ntactivity=ntactivity.drop('Number_of_Team_Members',axis=1)
ntactivity=ntactivity.fillna('Unknown')


In [0]:
ntactivity.isnull().any()

In [0]:
ntactivity = pd.concat([ntactivity.reset_index(drop=True), teamcount], axis=1)
ntactivity.head().T

In [0]:
ntactivity.isnull().any()

In [0]:
missing_team=ntactivity.Number_of_Team_Members.isnull()
ntactivity_missingteam=ntactivity[missing_team].copy()
ntactivity_missingteam #this will become our test data

In [0]:
print("shape of missign team : " + str(ntactivity_missingteam.shape) + "shape of original data : " + str(ntactivity.shape))

inverse the bool hence we have to use bitwise operator

In [0]:
having_team=~missing_team
having_team

In [0]:
ntactivity_havingteam=ntactivity[having_team].copy()
ntactivity_havingteam #this will become our test data

In [0]:
ntactivity_havingteam.isnull().values.any()

In [0]:
print("shape of missign team : " + str(ntactivity_missingteam.shape) + "shape of having data : " + str(ntactivity_havingteam.shape))

keep having data, split into train_test_validate, find best model and then impute accordingle

In [0]:
y=ntactivity_havingteam.Number_of_Team_Members
predictors=['Event_Chapter', 'Event_Category', 'Event_Type','Location', 'Corporate_Name', 'Revenue_Raised', 'City', 'State/Province',   'Primary_Connection_To_MS']
X=ntactivity_havingteam[predictors].copy()
X.shape

In [0]:
X_enc=pd.get_dummies(X)
X_enc.shape

In [0]:
X_missing=ntactivity_missingteam[predictors].copy()
X_missing_enc=pd.get_dummies(X_missing)
X_missing_enc.isnull().values.any()

Since not all the one hot encoded values are present in training set, we will extract columns from test and filter the train data. Also, we have to replace the nan values with 0 since they are not present

In [0]:
col_test=X_missing_enc.columns
X_enc=X_enc.loc[:,col_test]
nullvalues=X_enc.isnull()
X_enc=X_enc.fillna(0.0)
X_enc.isnull().values.any()

Since we have only 1 numeric column 'Revenue Raised' lets scale it

In [0]:
train_X, val_X, train_y, val_y = train_test_split(X_enc, y,random_state = 0)

In [0]:
my_pipeline_xgbr = make_pipeline(XGBRegressor(n_estimators=1000,learning_rate=0.2))
scores_xgbr = cross_val_score(my_pipeline_xgbr, train_X, train_y, scoring='neg_mean_squared_error')
my_pipeline_gbr = make_pipeline(GradientBoostingRegressor())
scores_gbr = cross_val_score(my_pipeline_gbr, train_X, train_y, scoring='neg_mean_squared_error')


In [0]:
print('New Mean Absolute Error %2f XGBR' %(-1 * scores_xgbr.mean()))
print('New Mean Absolute Error %2f GBR' %(-1 * scores_gbr.mean()))

In [0]:
my_model=GradientBoostingRegressor().fit(train_X,train_y)
val_predictions = my_model.predict(val_X)
print(mean_absolute_error(val_y, val_predictions))

In [0]:
predicted_NOTM = my_model.predict(X_missing_enc)


##########################################################################################################################

In [0]:
predicted_NOTM=predicted_NOTM.round().astype(int)
predicted_NOTM.dtype

In [0]:
X_missing_enc.shape

In [0]:
X_missing_enc['NOTM']=predicted_NOTM
notm_values=X_missing_enc['NOTM'].index
notm_values

In [0]:
ntactivity.iloc[notm_values,-1]=X_missing_enc['NOTM']
ntactivity.iloc[:,-1]
ntactivity.isnull().values.any()

# Analyzing Corporate Teams

In [0]:
corporate_participants=participantbike[participantbike.Team_Division=='Corporate'].copy()

In [0]:
filter_corporate=participantbike.Team_Division.str.contains("Corporate|Corporation")
corporate_participants=participantbike[filter_corporate]
corporate_participants.shape

In [0]:
corporate_participants_2016=corporate_participants[corporate_participants.Fiscal_Year==2016]
corporate_participants_2015=corporate_participants[corporate_participants.Fiscal_Year==2015]
corporate_participants_2014=corporate_participants[corporate_participants.Fiscal_Year==2014]
corporate_participants_2013=corporate_participants[corporate_participants.Fiscal_Year==2013]

In [0]:
print(str(corporate_participants_2016.shape) + "\t" + str(corporate_participants_2015.shape) + "\t" + str(corporate_participants_2014.shape) + "\t" + str(corporate_participants_2013.shape))

finding columns through which tables can be joined

In [0]:
donationsbike2016.columns


['Event_ID',  'Fiscal_Year','Gift_Amount($)',  'Gift_Type', 'Gift_Payment_Method','Donor_Employer','Participant_Goal($)','Additional_Gift_Amount($)',  'Original_Value_Transacted', 'Net_Transaction_Amount']

These columns are important from donation data

In [0]:
corporate_participants.columns


['Event_ID','Fiscal_Year','Total_of_All_Confirmed_Gifts($)','Participant_Employer','Registration_Type', 'Participant_Goal($)','Suggested_Participant_Goal($)']

These are important columns from participant data

In [0]:
#donationsbike2017_tomerge=donationsbike2017.loc[:,['Event_ID',  'Fiscal_Year','Gift_Amount($)',  'Gift_Type', 'Gift_Payment_Method','Donor_Employer','Participant_Goal($)','Additional_Gift_Amount($)',  'Original_Value_Transacted', 'Net_Transaction_Amount']]
donationsbike2016_tomerge=donationsbike2016.loc[:,['Event_ID',  'Fiscal_Year','Gift_Amount($)',  'Gift_Type', 'Gift_Payment_Method','Donor_Employer','Participant_Goal($)','Additional_Gift_Amount($)',  'Original_Value_Transacted', 'Net_Transaction_Amount']]
donationsbike2015_tomerge=donationsbike2015.loc[:,['Event_ID',  'Fiscal_Year','Gift_Amount($)',  'Gift_Type', 'Gift_Payment_Method','Donor_Employer','Participant_Goal($)','Additional_Gift_Amount($)',  'Original_Value_Transacted', 'Net_Transaction_Amount']]
donationsbike2014_tomerge=donationsbike2014.loc[:,['Event_ID',  'Fiscal_Year','Gift_Amount($)',  'Gift_Type', 'Gift_Payment_Method','Donor_Employer','Participant_Goal($)','Additional_Gift_Amount($)',  'Original_Value_Transacted', 'Net_Transaction_Amount']]
donationsbike2013_tomerge=donationsbike2013.loc[:,['Event_ID',  'Fiscal_Year','Gift_Amount($)',  'Gift_Type', 'Gift_Payment_Method','Donor_Employer','Participant_Goal($)','Additional_Gift_Amount($)',  'Original_Value_Transacted', 'Net_Transaction_Amount']]


In [0]:
corporate_participants_2016_tomerge=corporate_participants_2016.loc[:,['Event_ID','Fiscal_Year','Total_of_All_Confirmed_Gifts($)','Participant_Employer','Registration_Type', 'Participant_Goal($)','Suggested_Participant_Goal($)']]
corporate_participants_2015_tomerge=corporate_participants_2015.loc[:,['Event_ID','Fiscal_Year','Total_of_All_Confirmed_Gifts($)','Participant_Employer','Registration_Type', 'Participant_Goal($)','Suggested_Participant_Goal($)']]
corporate_participants_2014_tomerge=corporate_participants_2014.loc[:,['Event_ID','Fiscal_Year','Total_of_All_Confirmed_Gifts($)','Participant_Employer','Registration_Type', 'Participant_Goal($)','Suggested_Participant_Goal($)']]
corporate_participants_2013_tomerge=corporate_participants_2013.loc[:,['Event_ID','Fiscal_Year','Total_of_All_Confirmed_Gifts($)','Participant_Employer','Registration_Type', 'Participant_Goal($)','Suggested_Participant_Goal($)']]


In [0]:
from google.colab import files

corporate_participants_2016_tomerge.to_csv('corporate_participants_2016_tomerge.csv')
corporate_participants_2015_tomerge.to_csv('corporate_participants_2015_tomerge.csv')
corporate_participants_2014_tomerge.to_csv('corporate_participants_2014_tomerge.csv')
corporate_participants_2013_tomerge.to_csv('corporate_participants_2013_tomerge.csv')

donationsbike2016_tomerge.to_csv('donationsbike2016_tomerge.csv')
donationsbike2015_tomerge.to_csv('donationsbike2015_tomerge.csv')
donationsbike2014_tomerge.to_csv('donationsbike2014_tomerge.csv')
donationsbike2013_tomerge.to_csv('donationsbike2013_tomerge.csv')



In [0]:
files.download('corporate_participants_2016_tomerge.csv')

In [0]:
files.download('corporate_participants_2015_tomerge.csv')

In [0]:
files.download('corporate_participants_2014_tomerge.csv')

In [0]:
files.download('corporate_participants_2013_tomerge.csv')

In [0]:
files.download('donationsbike2016_tomerge.csv')

In [0]:
files.download('donationsbike2015_tomerge.csv')

In [0]:
files.download('donationsbike2014_tomerge.csv')

In [0]:
files.download('donationsbike2013_tomerge.csv')

#**Best Events**

In [0]:
eventsbike.head(3).T

In [0]:
df=eventsbike.loc[:,['Fiscal_Year','Event_ID','Event_Goal($)','Total_of_All_Confirmed_Gifts($)','Total_Online_Gifts($)','Total_Offline_Confirmed_Gifts($)','Total_From_Participant($)','Total_Not_From_Participant($)','Total_Team_Gifts($)','Total_Event_Gifts($)']]

In [0]:
df=df.sort_values(['Fiscal_Year','Total_of_All_Confirmed_Gifts($)'],ascending=[1, 0])

In [0]:
events_2013=df[df.Fiscal_Year==2013]
events_2014=df[df.Fiscal_Year==2014]
events_2015=df[df.Fiscal_Year==2015]
events_2016=df[df.Fiscal_Year==2016]
events_2017=df[df.Fiscal_Year==2017]

In [0]:
events_2013['Goal_Left']=events_2013.loc[:,'Event_Goal($)'] - events_2013.loc[:,'Total_of_All_Confirmed_Gifts($)']
events_2014['Goal_Left']=events_2014.loc[:,'Event_Goal($)'] - events_2014.loc[:,'Total_of_All_Confirmed_Gifts($)']
events_2015['Goal_Left']=events_2015.loc[:,'Event_Goal($)'] - events_2015.loc[:,'Total_of_All_Confirmed_Gifts($)']
events_2016['Goal_Left']=events_2016.loc[:,'Event_Goal($)'] - events_2016.loc[:,'Total_of_All_Confirmed_Gifts($)']
events_2017['Goal_Left']=events_2017.loc[:,'Event_Goal($)'] - events_2017.loc[:,'Total_of_All_Confirmed_Gifts($)']

In [0]:
events_2013['Goal_Achieved(%)']=100*(events_2013['Total_of_All_Confirmed_Gifts($)']/events_2013['Event_Goal($)'])
events_2014['Goal_Achieved(%)']=100*(events_2013['Total_of_All_Confirmed_Gifts($)']/events_2014['Event_Goal($)'])
events_2015['Goal_Achieved(%)']=100*(events_2013['Total_of_All_Confirmed_Gifts($)']/events_2015['Event_Goal($)'])
events_2016['Goal_Achieved(%)']=100*(events_2013['Total_of_All_Confirmed_Gifts($)']/events_2016['Event_Goal($)'])
events_2017['Goal_Achieved(%)']=100*(events_2013['Total_of_All_Confirmed_Gifts($)']/events_2017['Event_Goal($)'])


Since data is already sorted, lets extract top 5 events each year

**2013: Top 5 events by "confirmed gift" **

In [0]:
events_2013.nlargest(5,'Total_of_All_Confirmed_Gifts($)') 

**2013: Top 15 events by " % Goal Achieved" **

In [0]:
events_2013.nlargest(15,'Goal_Achieved(%)') 